In [ ]:
import synthingie as st

# Create a sound of raising pitch that repeats

In [ ]:
step_s = 60
duration = 180
voices = 7
base_freq = 22

In [ ]:
mod = st.Module(48000, int(48000/60))

In [ ]:
ramp = mod.ramp(0, voices / step_s) % voices     
freq = base_freq * 2 ** (1 + ramp)
pitch = mod.sin(freq)

In [ ]:
mod.render(pitch, duration).display()

# Create an envelope that fades out the start and end of the raise

In [ ]:
# create the envelope
amp = abs(mod.ramp(0.5, -1 / step_s) % 1 - 0.5) * 2
envelope = amp ** 3
    

In [ ]:
mod.render(envelope, duration).display()

# Mix them

In [ ]:
base_signal = envelope * pitch
mod.render(base_signal, duration).display()

# Mix with delayed copies so that the frequencies are integer multiples

In [ ]:
final_sound = base_signal
for v in range(1, voices):
    final_sound += base_signal.fixed_delay(v * step_s / voices) 
mod.render(final_sound, duration).display()

# Refactor everything into one re usable function

In [ ]:
def shepard(base_pitch):
    duration = step_s * 5

    # create the pitch
    ramp = mod.ramp(0, voices / step_s) % voices     
    freq = base_pitch * 2 ** (1 + ramp)
    pitch = mod.sin(freq)
    
    # create the envelope
    amp = abs(mod.ramp(0.5, -1 / step_s) % 1 - 0.5) * 2
    amp = amp ** 3
    
    # mixit
    base_signal = amp * pitch

    # create voices
    final_sound = base_signal
    for v in range(1, voices):
        final_sound += base_signal.fixed_delay(v * step_s / voices) 
    
    return final_sound * (1/20.)

# Add a fake chorus to hide it

In [ ]:
fhz = 22
s = shepard(base_freq) + shepard(fhz * 1.005) * 0.5 + shepard(fhz * 0.995) * 0.5
mod.render(s, 180).display()